In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt


In [2]:
from package.src.loading import DataLoader
loader = DataLoader('./data/kidney_disease.csv',test_size=0.1)
loader.load(target_column="classification")

Dataset has been loaded in split into training and test data


In [3]:
X_train, y_train,X_test,y_test = loader.X_train, loader.y_train, loader.X_test, loader.y_test
from package.src.preprocessing import FeatureSelector
from package.src.preprocessing import DataPreprocessor, DataNormaliser
pp = DataPreprocessor(missing_data_strategy= "mean", categorical_encoding= "label")
fs = FeatureSelector()
dn = DataNormaliser()
X_train,y_train= pp.fit_transform(X_train,y_train)
X_test,y_test=pp.transform(X_test,y_test)

Data Preprocessing for the train data is starting
We infer missing data
Inference of missing data is Done
We correct typos in the data
Typo detected on column : dm betwen  	yes and   yes
Typo corrected
Typo detected on column : dm betwen  	yes and  yes
Typo corrected
Typo detected on column : dm betwen   yes and  yes
Typo corrected
Typo detected on column : dm betwen  	no and  no
Typo corrected
Typo detected on column : cad betwen  	no and  no
Typo corrected
Typo detected on column : classification betwen  ckd	 and  ckd
Typo corrected
Typos correction is Done
We encode the categorical columns
Columns encoding is Done
Train Data preprocessing is done 
Data Preprocessing for the test data is starting
We infer missing data
Inference of missing data is Done
We correct typos in the data
Correction typos for test dataset for  rbc
Correction typos for test dataset for  pc
Correction typos for test dataset for  pcc
Correction typos for test dataset for  ba
Correction typos for test dataset for

In [4]:
dn.fit(X_train)
X_train=dn.transform(X_train)
X_test=dn.transform(X_test)
fs.fit(X_train)

25 componenents (PCA)


In [5]:
X_train=fs.transform(X_train)
X_test=fs.transform(X_test)

data reduced to 20 dimensions
data reduced to 20 dimensions


In [6]:
from sklearn.linear_model import LogisticRegression
from package.src.modeling import Model
model = Model(LogisticRegression())

In [8]:
model.model.fit(X_train,y_train)
y_p = model.model.predict(X_test)

In [10]:
model.f1_score_support(y_test,y_p)

,Precision,Recall,F1_score
class 0,1,1,1.0
class 1,1,1,1.0


In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np

class Model():
    def __init__(self, model):
        if model not in ["LogisticRegression", "DecisionTrees","KNN","SVM","RandomForest"]:
            raise Exception("Model not supported \n choose in list ....  ")
        self.model = model
        self.skmodel = None
        self.params = None
        if self.model == "LogisticRegression":
            self.skmodel = LogisticRegression()
            self.params = {'C': [0.001, .009, 0.01, .09, 1, 5, 10, 25]}
        elif self.model == "DecisionTrees":
            self.skmodel = DecisionTreeClassifier()
            self.params ={'min_samples_split': range(10, 500, 20), 'max_depth': range(1, 20, 2), 'criterion': ['gini', 'entropy']}
        elif self.model == "KNN":
            self.skmodel = KneighborsClassifier()
            self.params ={'n_neighbors': [4, 5, 6, 7], 'leaf_size': [1, 3, 5], 'weights': ['uniform', 'distance']}
        elif self.model == "SVM":
            self.skmodel = SVC()
            self.params = {}
        elif self.model == "RandomForest":
            self.skmodel = RandomForestClassifier()
            self.params = {}
            
    def fit(self,X,Y,params_dic): 
        if self.model == "LogisticRegression":
            self.skmodel = LogisticRegression(params_dic)
            self.skmodel.fit(X,Y)
    def predict(self,X):
        return self.skmodel.predict(X)

    def cross_validation_score(self, X_train: np.array, y_train: np.array, metric = "accuracy", k=10):
        evaluation = []
        for i in range(k):
            X_test_fold = X_train[int((i / k) * len(X_train)):int(((i + 1) / k) * len(X_train))]
            y_test_fold = y_train[int((i / k) * len(y_train)):int(((i + 1) / k) * len(y_train))]
            X_train_fold = np.concatenate(
                (X_train[:int((i / k) * len(X_train))], X_train[int(((i + 1) / k) * len(X_train)):]))
            y_train_fold = np.concatenate(
                (y_train[:int((i / k) * len(y_train))], y_train[int(((i + 1) / k) * len(y_train)):]))
            self.skmodel.fit(X_train_fold, y_train_fold)
            y_pred = self.skmodel.predict(X_test_fold)
            if metric == "accuracy":
                evaluation.append(self.accuracy(y_test_fold, y_pred))
            else : 
                evaluation.append(self.f1_score(y_test_fold, y_pred))

                
            evaluation.append(self.f1_score(y_test_fold, y_pred))
        return np.mean(evaluation)



    @staticmethod
    def f1_score(y_true, y_predict):
        p = 0
        r = 0
        n_classes = y_true.nunique()
        for i in range(n_classes):
            tp = sum(((y_true == i) & (y_predict == i)))
            fp = sum(((y_true != i) & (y_predict == i)))
            fn = sum(((y_true == i) & (y_predict != i)))
            p += tp / (tp + fp)
            r += tp / (tp + fn)
        p = p / n_classes
        r = r / n_classes
        return 2 * p * r / (p + r)
    @staticmethod    
    def accuracy (y_true, y_predict):
        good_predictions = np.sum((y_true == y_predict)*1)
        return good_predictions /len(y_true)



In [ ]:
model